#### 1. Connect to DB

In [77]:
import psycopg2

host = "140.112.12.188"
dbname = "dbms_final"
user = "andychang"
password = "1234"
#sslmode = "allow"

#conn_string = "host='140.112.12.188' user='andychang' dbname='dbms_final' password='1234' port='5432'"
# 以 super user 身份登入
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established



##### 2. Create new role 'role1'

In [53]:
cur = conn.cursor()
try: 
    cur.execute("CREATE ROLE role1;")
    cur.execute("SELECT rolname FROM pg_roles;")


except psycopg2.Error as e:
    print(f"Error : {e}")
cur.close()

('pg_database_owner',)
('pg_read_all_data',)
('pg_write_all_data',)
('pg_monitor',)
('pg_read_all_settings',)
('pg_read_all_stats',)
('pg_stat_scan_tables',)
('pg_read_server_files',)
('pg_write_server_files',)
('pg_execute_server_program',)
('pg_signal_backend',)
('pg_checkpoint',)
('pg_use_reserved_connections',)
('pg_create_subscription',)
('postgres',)
('andychang',)
('role1',)


增加 index 以確保 Qury Optimizer 先執行 user filter 再執行 RLS policy 

In [59]:
try:
    cur.execute("CREATE INDEX index_user_name ON RLS_test (user_name);")
    conn.commit()
except psycopg2.Error as e:
    print(f"Error : {e}")
    conn.rollback()

('index_user_name', 'CREATE INDEX index_user_name ON public.rls_test USING btree (user_name)')
('rls_test_pkey', 'CREATE UNIQUE INDEX rls_test_pkey ON public.rls_test USING btree (id)')


##### 3. Create Patient Table and Insert Data 

In [78]:
import random
import string
import csv
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS Patient;")
cursor.execute("CREATE TABLE Patient (PK serial PRIMARY KEY, name VARCHAR(50), ID_Number VARCHAR(50) , Age INT, Site_ID INT);")
# cursor.execute("GRANT ALL PRIVILEGES ON RLS_test TO PUBLIC;")
# cursor.execute("ALTER TABLE RLS_test ENABLE ROW LEVEL SECURITY;")
# cursor.execute("CREATE POLICY account_managers ON RLS_test USING(user_name=current_user);")
# cursor.execute("INSERT INTO Patient (name, ID_Number, Age, Site_ID) VALUES (%s, %s, %s, %s);", ("Anthony Moore", '9788453650', 24, 0))
conn.commit()
cursor.execute("SELECT * FROM Patient;")
print(cursor.fetchall())

def generate_unique_id(existing_ids, length=10):
    # generate 唯一的 ID_Number
    while True:
        new_id = ''.join(random.choices(string.digits, k=length))
        if new_id not in existing_ids:
            existing_ids.add(new_id)
            return new_id

# 生資料並寫入 CSV 檔案
csv_file_path = 'data.csv'
names = ["Anthony Moore", "John Doe", "Jane Smith"]
ages = [24, 30, 35, 40, 45]
site_ids = [0, 1]
num_records = 10000

existing_ids = set()

with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'ID_Number', 'Age', 'Site_ID']) # csv header 
    for _ in range(num_records):
        name = random.choice(names)
        age = random.choice(ages)
        site_id = random.choice(site_ids)
        id_number = generate_unique_id(existing_ids)
        writer.writerow([name, id_number, age, site_id])

try:
    cur = conn.cursor()
    # 將 CSV 檔案中的資料載入到資料庫
    with open(csv_file_path, 'r') as csvfile:
        cur.copy_expert("COPY Patient (name, ID_Number, Age, Site_ID) FROM STDIN WITH CSV HEADER", csvfile)
    conn.commit()
    print(f"Successfully inserted {num_records} records into the Patient table using COPY command.")

except psycopg2.Error as e:
    print(f"Error: {e}")
    if conn:
        conn.rollback()

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

[]
Successfully inserted 10000 records into the Patient table using COPY command.
